# Lab 2: demo 📓

The purpose of this lab is to add the ability to decouple parsing/writing from creating the `RNA_Molecule` object. This is an encapsulatd step that will not change the way the user would interact with this utility, but rather make the interface more flexible and less susceptible to errors coming from changes in the future.

In [2]:
# -- necessary setup

import os,sys
sys.path.append(os.path.abspath('../src'))

from IO.RNA_IO import RNA_IO
from utils import pathify_pdb

## Reading from PDB File

In [4]:
rna_io=RNA_IO()
pdb_path=pathify_pdb('7EAF')
mol=rna_io.read(pdb_path, "PDB",array=False)
len(mol.get_models()[0].get_chains()['A'].get_residues())

>> initializing new species: CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS <<


94

In [24]:
list(mol.get_models()[0].get_chains()['A'].get_residues()[9].get_atoms().keys())[0][1]=='' #if has alt will return false, else (occupancy is 1) will return true
mol.get_models()[0].get_chains()['A'].get_residues()[9].get_atoms()[('P', 'A')].occupancy

0.44

In [2]:
#here array is defaulted to true

rna_io=RNA_IO()
pdb_path=pathify_pdb('7EAF')
mol=rna_io.read(pdb_path, "PDB",array=True)
mol.shape
# mol

>> initializing new species: CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS <<


(1, 95, 3)

In [28]:
rna_molecule=mol

In [59]:
residue9.get_atoms()[('P', 'A')].occupancy
residue9.type.name
residue9.get_atoms().items()
residue9.position
atom1.element.name


'O'

In [69]:
for i,j in enumerate(mol.get_models()):
    print(type(i),type(j))

<class 'int'> <class 'int'>


In [72]:
def extract_atoms_from_rna(rna_molecule):
    atoms_list = []
    model = rna_molecule.get_models()[0]  # Access first model

    for model_num,_ in enumerate(rna_molecule.get_models()):  # Iterate over models
        model=rna_molecule.get_models()[_] # --model object from dict key
    
        for chain in model.get_chains().values():  # Iterate over chains
            for residue in chain.get_residues().values():  # Iterate over residues
                for atom_key, atom in residue.get_atoms().items():  # Iterate over atoms
                    atom_id, alt_id = atom_key  # Unpack atom key
                    
                    atom_data = {
                        "id": str(len(atoms_list) + 1),  # Assign a sequential ID
                        "B_iso_or_equiv": str(atom.temp_factor),
                        "Cartn_x": str(atom.x),
                        "Cartn_y": str(atom.y),
                        "Cartn_z": str(atom.z),
                        "auth_asym_id": chain.id,
                        "auth_atom_id": atom_id,
                        "auth_comp_id": residue.type.name,
                        "auth_seq_id": str(residue.position),
                        "group_PDB": "ATOM", # assuming afor now that all are ATOM records
                        "label_alt_id": None if alt_id == "" else alt_id,
                        "label_asym_id": chain.id,
                        "label_atom_id": atom_id,
                        "label_comp_id": residue.type.name,
                        "label_entity_id": "1",  # should be one entity per xml for now
                        "label_seq_id": str(residue.position),
                        "occupancy": str(atom.occupancy),
                        "pdbx_PDB_model_num": model_num+1,
                        "type_symbol": atom.element.name
                    }

                    #if atom.occupancy == 1:
                    #    atom_data["label_alt_id"] = {"xsi:nil": "true"}
                    # --will try adding it in xml formatting
                    
                    atoms_list.append(atom_data)
    
    return atoms_list


In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def wrap_str_to_xml(s,name='pdbml_output.xml'):
    with open(name, "w") as f:
        f.write(s)


def pretty_print_xml(func):
    def wrapper(*args, **kwargs):
        # Call the original function and get the XML string
        xml_string = func(*args, **kwargs)
        
        # Parse the XML string
        dom = minidom.parseString(xml_string)
        
        # Pretty print the XML
        pretty_xml = dom.toprettyxml(indent="  ")
        
        return pretty_xml
    return wrapper

@pretty_print_xml
def create_xml_from_molecule(rna_molecule):
    root = ET.Element("PDBx:datablock", {
        "datablockName": rna_molecule.entry_id,
        "xmlns:PDBx": "http://pdbml.pdb.org/schema/pdbx-v50.xsd",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "xsi:schemaLocation": "http://pdbml.pdb.org/schema/pdbx-v50.xsd pdbx-v50.xsd"
    })

    atom_site_category = ET.SubElement(root, "PDBx:atom_siteCategory")

    atoms = extract_atoms_from_rna(rna_molecule)

    for atom in atoms:
        atom_site = ET.SubElement(atom_site_category, "PDBx:atom_site", {"id": atom["id"]})

        for key, value in atom.items():
            if key == "id":
                continue
            element = ET.SubElement(atom_site, f"PDBx:{key}")
            if value is None:
                element.set("xsi:nil", "true")
            else:
                element.text = str(value)  # Convert value to string

    # tree = ET.ElementTree(root)
    # with open("pdbml_output.xml", "wb") as f:
    #     tree.write(f, encoding="utf-8", xml_declaration=True)

    xml_string = ET.tostring(root, encoding="unicode", method="xml")
    return xml_string



s=create_xml_from_molecule(rna_molecule)
wrap_str_to_xml(s)

In [76]:
from xml.dom.minidom import parseString
import xml.etree.ElementTree as ET

def pretty_print_xml(xml_string):
    dom = parseString(xml_string)
    return dom.toprettyxml()

xml_string = """<root><child><subchild>data</subchild></child></root>"""
pretty_xml = pretty_print_xml(xml_string)
print(pretty_xml)

<?xml version="1.0" ?>
<root>
	<child>
		<subchild>data</subchild>
	</child>
</root>



In [73]:
extract_atoms_from_rna(mol)

[{'id': '1',
  'B_iso_or_equiv': '110.87',
  'Cartn_x': '-9.698',
  'Cartn_y': '3.426',
  'Cartn_z': '-31.854',
  'auth_asym_id': 'A',
  'auth_atom_id': 'OP3',
  'auth_comp_id': 'G',
  'auth_seq_id': '1',
  'group_PDB': 'ATOM',
  'label_alt_id': None,
  'label_asym_id': 'A',
  'label_atom_id': 'OP3',
  'label_comp_id': 'G',
  'label_entity_id': '1',
  'label_seq_id': '1',
  'occupancy': '1.0',
  'pdbx_PDB_model_num': 1,
  'type_symbol': 'O'},
 {'id': '2',
  'B_iso_or_equiv': '107.82',
  'Cartn_x': '-8.782',
  'Cartn_y': '4.433',
  'Cartn_z': '-32.44',
  'auth_asym_id': 'A',
  'auth_atom_id': 'P',
  'auth_comp_id': 'G',
  'auth_seq_id': '1',
  'group_PDB': 'ATOM',
  'label_alt_id': None,
  'label_asym_id': 'A',
  'label_atom_id': 'P',
  'label_comp_id': 'G',
  'label_entity_id': '1',
  'label_seq_id': '1',
  'occupancy': '1.0',
  'pdbx_PDB_model_num': 1,
  'type_symbol': 'P'},
 {'id': '3',
  'B_iso_or_equiv': '106.7',
  'Cartn_x': '-8.042',
  'Cartn_y': '5.072',
  'Cartn_z': '-31.321',


In [54]:
model=rna_molecule.get_models()[0]
chain=model.get_chains()['A']
residue9=chain.get_residues()[9]
residue1=chain.get_residues()[1]
atom1=residue1.get_atoms()[('OP3', '')]
atom1.temp_factor #this will be the B_iso_or_equiv
atom1.x,atom1.y,atom1.z #this will be the coordinates
atom1.occupancy #this will be the occupancy
atom1.residue.type.name #this is how to access residue
atom1.residue.chain.id #this is how to access chain
chain.model.id #this is how to access model id

0

In [29]:
rna_molecule.entry_id

'7EAF'

In [ ]:
import xml.etree.ElementTree as ET
root = ET.Element("PDBx:datablock", {
    "datablockName": rna_molecule.entry_id,
    "xmlns:PDBx": "http://pdbml.pdb.org/schema/pdbx-v50.xsd",
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsi:schemaLocation": "http://pdbml.pdb.org/schema/pdbx-v50.xsd pdbx-v50.xsd"
})
atom_site_category = ET.SubElement(root, "PDBx:atom_siteCategory")


##  Writing the Output to a PDB-like format File

In [5]:
rna_io.write(mol, "demo.xml",'PDBML')

# with open("demo.pdb", "r") as f:
#     print(f.read())

RNA molecule written to demo.xml


To check the file content, check the file [`demo.pdb`](./demo.pdb) in the same directory as this notebook (demo/)

##  Reading from the Written File

In [16]:
mol1=rna_io.read("demo.pdb", "PDB")

> note: Species with this name already exists, will return the same instance
RNA_Molecule 7EAF already exists in the species CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS; not added again


##  Writing again the Content to another File 
To verify the integrity of the reading and writing functions, the content will be written to a second file. By comparing the two files, we can ensure that the content is identical, confirming that the functions are correctly implemented and that the PDB format is followed accurately.

In [20]:
rna_io.write(mol1, "demo1.pdb",'PDB')

# with open("demo1.pdb", "r") as f:
#     print(f.read())

RNA molecule written to demo1.pdb


Instead of printing, checkout the file content in [`demo1.pdb`](./demo1.pdb) file found in the current directory.

## Coarse-Grained 3D Representation



In [18]:
cg_mol=rna_io.read(pdb_path, "PDB", coarse_grained=True, atom_name="C1'") #atom_name is optional, default is "C1'"
rna_io.write(cg_mol, "demo_cg.pdb",'PDB')

> note: Species with this name already exists, will return the same instance
RNA_Molecule 7EAF already exists in the species CALDANAEROBACTER SUBTERRANEUS SUBSP. TENGCONGENSIS; not added again
RNA molecule written to demo_cg.pdb


## Reading other PDB Entries

In [19]:
from utils import get_pdb_ids_from_fam
from Families.family import Family

pdb_ids=get_pdb_ids_from_fam('SAM')
molecules=[]
fam=Family(id='RF01510',name='2dG-I ')

for pdb_id in pdb_ids:
    rna_io=RNA_IO()
    rna=rna_io.read(pathify_pdb(pdb_id),format='PDB')
    molecules.append(rna)


Family with this id already exists, will link it to the existing family
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK6 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK3 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FKF already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 4KQY already exists in the species BACILLUS SUBTILIS; not added again
> note: Species with this name already exists, will return the same instance
RNA_Molecule 5FK4 already exists in the species THERMOANAEROBACTER TENGCONGENSIS; not added again
> note: Species with this name already exists, will return the same 

In [12]:
print(f'number of molecules in 2dG-I family: {len(molecules)}')
for x in molecules:
    print(f'type of item {x} in molecules: {type(x)}')

number of molecules in 2dG-I family: 29
type of item ID: 5FK6 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 3GX2 Experiment: X-RAY DIFFRACTION None in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FK3 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FKF Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 4KQY Experiment: X-RAY DIFFRACTION BACILLUS SUBTILIS species in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 3GX6 Experiment: X-RAY DIFFRACTION None in molecules: <class 'Structure.RNA_Molecule.RNA_Molecule'>
type of item ID: 5FK4 Experiment: X-RAY DIFFRACTION THERMOANAEROBACTER TENGCONGENSIS species in molecules: <class 'Structure.RNA_